In [1]:
#importing libraries
import numpy as np

In [2]:
# downloading MNIST datasets
def download_dataset():
    
    from urllib.request import urlretrieve
    from os.path import isfile, isdir
    from tqdm import tqdm
    import tarfile


    cifar10_dataset_folder_path = 'cifar-10-batches-py'
    tar_gz_path = 'cifar-10-python.tar.gz'

    class DLProgress(tqdm):
        last_block = 0

        def hook(self, block_num=1, block_size=1, total_size=None):
            self.total = total_size
            self.update((block_num - self.last_block) * block_size)
            self.last_block = block_num

    if not isfile(tar_gz_path):
        with DLProgress(unit='B', unit_scale=True, miniters=1, desc='CIFAR-10 Dataset') as pbar:
            urlretrieve(
                'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
                tar_gz_path,
                pbar.hook)

    if not isdir(cifar10_dataset_folder_path):
        with tarfile.open(tar_gz_path) as tar:
            tar.extractall()
            tar.close()

In [3]:
#preprocess the images dataset
cifar10_dataset_folder_path = 'cifar-10-batches-py'

def one_hot_encoding(x):
    
    n_values = 10
    return np.eye(n_values)[x]

def normalize(x):
    
    return x/255

def load_images_labels():
    
    import pickle
    from sklearn.utils import shuffle
    
    batch_ids = 5
    train_validation_dataset = []
    
    for batch_id in range(1, batch_ids + 1):
        with open(cifar10_dataset_folder_path + "/data_batch_" + str(batch_id), mode='rb' ) as file:
            batch = pickle.load(file, encoding='latin1')
            for data, label in zip(batch['data'], batch['labels']):
                train_validation_dataset.append((normalize(data), one_hot_encoding(label)))
    
    test_dataset = []
    with open(cifar10_dataset_folder_path + "/test_batch", mode='rb') as file:
        for data, label in zip(batch['data'], batch['labels']):
            test_dataset.append((normalize(data), one_hot_encoding(label)))
            
    return shuffle(shuffle(train_validation_dataset)), test_dataset

def extract_images_labels(images_labels):
    
    images = np.array([x[0] for x in images_labels])
    labels = np.array([x[1] for x in images_labels])
    
    return images, labels

In [4]:
download_dataset()
train_val_dataset, test_dataset = load_images_labels()
train_images, train_labels = extract_images_labels(train_val_dataset)
test_images, test_labels = extract_images_labels(test_dataset)

In [23]:
class NeuralNetwork:
    
    def __init__(self, inputs, hidden_layer1, hidden_layer2, outputs, batch_size=10000, learning_rate=0.1, epochs=100):
        
        self.input_nodes = inputs
        self.hidden_layer1_nodes = hidden_layer1
        self.hidden_layer2_nodes = hidden_layer2
        self.output_nodes = outputs
        
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        
        self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5, (self.input_nodes, self.hidden_layer1_nodes))
        self.weights_hidden_to_hidden = np.random.normal(0.0, self.hidden_layer1_nodes**-0.5, (self.hidden_layer1_nodes, self.hidden_layer2_nodes))
        self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_layer2_nodes**-0.5, (self.hidden_layer2_nodes, self.output_nodes))

        
    def sigmoid(self, x):
        
        return 1/(1 + np.exp(-x))
    
    def sigmoid_prime(self, x):
        
        return self.sigmoid(x) * (1 - self.sigmoid(x))
        
    def train_complete(self, images, targets):
        
        num_records = len(images)
        for i in range(self.epochs):
            
            delta_weights_input_to_hidden = np.zeros(self.weights_input_to_hidden.shape)
            delta_weights_hidden_to_hidden = np.zeros(self.weights_hidden_to_hidden.shape)
            delta_weights_hidden_to_output = np.zeros(self.weights_hidden_to_output.shape)
            
            # forward pass
            weights_hidden_layer1_in, weights_hidden_layer1_out, weights_hidden_layer2_in, \
                    weights_hidden_layer2_out, output_layer_in, final_outputs = self.forward_pass(images)
            
            # backpropagation
            delta_weights_input_to_hidden, delta_weights_hidden_to_hidden, \
                        delta_weights_hidden_to_output = self.backpropagation(images, targets, weights_hidden_layer1_in, \
                        weights_hidden_layer1_out, weights_hidden_layer2_in, weights_hidden_layer2_out, output_layer_in, final_outputs, \
                        delta_weights_input_to_hidden, delta_weights_hidden_to_hidden, delta_weights_hidden_to_output)
            
            # Updating the weights
            self.update_weights(delta_weights_input_to_hidden, delta_weights_hidden_to_hidden, delta_weights_hidden_to_output, num_records)
            
            # Calculating the error
            train_error = self.calculate_error(final_outputs, targets)
            validation_error, validation_accuracy = self.calculate_validation_stats()
            print("Epoch {0}, Training Error {1:0.3f}, Validation Error {2:0.3f}, Validation Accuracy {3:0.3f}".format(i, train_error, validation_error, validation_accuracy))
            
                
    def train_batches(self, images, targets):
        
        num_records = len(images)
        for i in range(self.epochs):
            
            #computing the batches
            for j in range(0, num_records, self.batch_size):
                
                batch_images = images[j: j + self.batch_size]
                batch_targets = targets[j: j + self.batch_size]
                
                delta_weights_input_to_hidden = np.zeros(self.weights_input_to_hidden.shape)
                delta_weights_hidden_to_hidden = np.zeros(self.weights_hidden_to_hidden.shape)
                delta_weights_hidden_to_output = np.zeros(self.weights_hidden_to_output.shape)

                # do the forward pass
                weights_hidden_layer1_in, weights_hidden_layer1_out, weights_hidden_layer2_in, \
                        weights_hidden_layer2_out, output_layer_in, final_outputs = self.forward_pass(batch_images)
                
                # do the backpropagation
                delta_weights_input_to_hidden, delta_weights_hidden_to_hidden, \
                        delta_weights_hidden_to_output = self.backpropagation(batch_images, batch_targets, weights_hidden_layer1_in, \
                        weights_hidden_layer1_out, weights_hidden_layer2_in,weights_hidden_layer2_out, output_layer_in, final_outputs, \
                        delta_weights_input_to_hidden, delta_weights_hidden_to_hidden, delta_weights_hidden_to_output)
                
                # Update the weights
                self.update_weights(delta_weights_input_to_hidden, delta_weights_hidden_to_hidden, delta_weights_hidden_to_output, num_records)
                
                # Calculating the error
                train_error = self.calculate_error(final_outputs, batch_targets)
                validation_error, validation_accuracy = self.calculate_validation_stats()
                print("Epoch {0}, Training Error {1:0.3f}, Validation Error {2:0.3f}, Validation Accuracy {3:0.3f}".format(i, \
                                                                    train_error, validation_error, validation_accuracy))
                
                                      
    def forward_pass(self, features):
        
        weights_hidden_layer1_in = np.dot(features, self.weights_input_to_hidden)
        weights_hidden_layer1_out = self.sigmoid(weights_hidden_layer1_in)
        
        weights_hidden_layer2_in = np.dot(weights_hidden_layer1_out, self.weights_hidden_to_hidden)
        weights_hidden_layer2_out = self.sigmoid(weights_hidden_layer2_in)
        
        output_layer_in = np.dot(weights_hidden_layer2_out, self.weights_hidden_to_output)
        final_outputs = self.sigmoid(output_layer_in)
        
        return weights_hidden_layer1_in, weights_hidden_layer1_out, weights_hidden_layer2_in, \
                weights_hidden_layer2_out, output_layer_in, final_outputs
    
    def backpropagation(self, features, labels, weights_hidden_layer1_in, weights_hidden_layer1_out, weights_hidden_layer2_in, \
                              weights_hidden_layer2_out, output_layer_in, final_outputs, \
                              delta_weights_input_to_hidden, delta_weights_hidden_to_hidden, delta_weights_hidden_to_output):

        """ 
        Note: Error in output layer is the difference of predicted outputs to actual outputs
              Error term would be the derivate of the activation function used in that layer. 
              Eg: x -> y -> z=f(y) here the activation function would be z. so error term would be error * f'(y)
                  in case of sigmoid error * sig(y) * (1- sig(y)) ; here z = sig(y)
                  so we can directly write error * z * (1-z) 
        """
        
        error = labels - final_outputs
        output_error_term = error * self.sigmoid_prime(output_layer_in)
        
        hidden_layer2_error = np.dot(output_error_term, self.weights_hidden_to_output.T)
        # we can also use the sigmoid prime method. here we are using direcly output of hidden layer
        hidden_layer2_error_term = hidden_layer2_error * weights_hidden_layer2_out * (1 - weights_hidden_layer2_out) 
        
        hidden_layer1_error = np.dot(hidden_layer2_error_term, self.weights_hidden_to_hidden.T)
        # third way of writing the error term
        hidden_layer1_error_term = hidden_layer1_error * self.sigmoid(weights_hidden_layer1_in) * (1 - self.sigmoid(weights_hidden_layer1_in))
        
        delta_weights_input_to_hidden += np.dot( features.T,  hidden_layer1_error_term )
        delta_weights_hidden_to_hidden += np.dot(weights_hidden_layer1_out.T , hidden_layer2_error_term )
        delta_weights_hidden_to_output += np.dot(weights_hidden_layer2_out.T , output_error_term)
        
        return delta_weights_input_to_hidden, delta_weights_hidden_to_hidden, delta_weights_hidden_to_output
    
    def update_weights(self, delta_weights_input_to_hidden, delta_weights_hidden_to_hidden, delta_weights_hidden_to_output, num_records):
        
        self.weights_input_to_hidden += self.learning_rate * delta_weights_input_to_hidden / num_records
        self.weights_hidden_to_hidden += self.learning_rate * delta_weights_hidden_to_hidden / num_records
        self.weights_hidden_to_output +=  self.learning_rate * delta_weights_hidden_to_output / num_records
    
    def calculate_error(self, final_outputs, labels):
        error = (labels - final_outputs)**2
        mean_error = np.sum(error)/labels.shape[0]
        
        return mean_error
                      
    def calculate_validation_stats(self):
        
        validation_results = self.test(self.validation_features)
        validation_error = (self.validation_targets - validation_results)**2
        mean_validation_error = np.sum(validation_error)/self.validation_targets.shape[0]
        
        check_max = (validation_results == validation_results.max(axis=1)[:,None])
        check_greater_than_80 = validation_results >= 0.8
        both_conditions = check_max & check_greater_than_80
        
        round_validation_results = both_conditions.astype(int)
        labels_int = self.validation_targets.astype(int)
        num_errors = np.count_nonzero(np.sum(round_validation_results - labels_int, axis=1))
        correct = self.validation_targets.shape[0] - num_errors
        validation_accuracy = correct / self.validation_targets.shape[0]
        
        return mean_validation_error, validation_accuracy
        
    def test(self, features):
        
        weights_hidden_layer1_in = np.dot(features, self.weights_input_to_hidden)
        weights_hidden_layer1_out = self.sigmoid(weights_hidden_layer1_in)
        
        weights_hidden_layer2_in = np.dot(weights_hidden_layer1_out, self.weights_hidden_to_hidden)
        weights_hidden_layer2_out = self.sigmoid(weights_hidden_layer2_in)
        
        output_layer_in = np.dot(weights_hidden_layer2_out, self.weights_hidden_to_output)
        final_outputs = self.sigmoid(output_layer_in)
        
        return final_outputs
    
    def run(self, features, targets):
        
        import timeit

        from sklearn.model_selection import train_test_split
        X_train, X_val, y_train, y_val = train_test_split( features, targets, test_size=0.2, random_state=42)
        
        self.validation_features = X_val
        self.validation_targets = y_val

#         start = timeit.timeit()
#         self.train_complete(X_train, y_train)
#         end = timeit.timeit()
#         print("Time for training Neural Network using Complete batch: {0}".format(end - start)) 
        
        start = timeit.timeit()
        self.train_batches(X_train, y_train)
        end = timeit.timeit()
        print("Time for training Neural Network using mini batches: {0}".format(end - start)) 
        
       

In [24]:
network = NeuralNetwork(3072, 128, 32, 10)

In [25]:
network.run(train_images, train_labels)

Epoch 0, Training Error 2.345, Validation Error 2.293, Validation Accuracy 0.000
Epoch 0, Training Error 2.293, Validation Error 2.243, Validation Accuracy 0.000
Epoch 0, Training Error 2.243, Validation Error 2.195, Validation Accuracy 0.000
Epoch 0, Training Error 2.194, Validation Error 2.149, Validation Accuracy 0.000
Epoch 1, Training Error 2.149, Validation Error 2.104, Validation Accuracy 0.000
Epoch 1, Training Error 2.104, Validation Error 2.061, Validation Accuracy 0.000
Epoch 1, Training Error 2.061, Validation Error 2.020, Validation Accuracy 0.000
Epoch 1, Training Error 2.019, Validation Error 1.980, Validation Accuracy 0.000
Epoch 2, Training Error 1.981, Validation Error 1.942, Validation Accuracy 0.000
Epoch 2, Training Error 1.942, Validation Error 1.906, Validation Accuracy 0.000
Epoch 2, Training Error 1.906, Validation Error 1.870, Validation Accuracy 0.000
Epoch 2, Training Error 1.869, Validation Error 1.837, Validation Accuracy 0.000
Epoch 3, Training Error 1.83

Epoch 25, Training Error 0.986, Validation Error 0.985, Validation Accuracy 0.000
Epoch 25, Training Error 0.984, Validation Error 0.983, Validation Accuracy 0.000
Epoch 25, Training Error 0.983, Validation Error 0.981, Validation Accuracy 0.000
Epoch 25, Training Error 0.981, Validation Error 0.980, Validation Accuracy 0.000
Epoch 26, Training Error 0.980, Validation Error 0.978, Validation Accuracy 0.000
Epoch 26, Training Error 0.978, Validation Error 0.977, Validation Accuracy 0.000
Epoch 26, Training Error 0.977, Validation Error 0.975, Validation Accuracy 0.000
Epoch 26, Training Error 0.975, Validation Error 0.974, Validation Accuracy 0.000
Epoch 27, Training Error 0.974, Validation Error 0.972, Validation Accuracy 0.000
Epoch 27, Training Error 0.972, Validation Error 0.971, Validation Accuracy 0.000
Epoch 27, Training Error 0.971, Validation Error 0.970, Validation Accuracy 0.000
Epoch 27, Training Error 0.969, Validation Error 0.968, Validation Accuracy 0.000
Epoch 28, Traini

Epoch 49, Training Error 0.917, Validation Error 0.917, Validation Accuracy 0.000
Epoch 50, Training Error 0.917, Validation Error 0.917, Validation Accuracy 0.000
Epoch 50, Training Error 0.917, Validation Error 0.917, Validation Accuracy 0.000
Epoch 50, Training Error 0.917, Validation Error 0.916, Validation Accuracy 0.000
Epoch 50, Training Error 0.916, Validation Error 0.916, Validation Accuracy 0.000
Epoch 51, Training Error 0.916, Validation Error 0.916, Validation Accuracy 0.000
Epoch 51, Training Error 0.916, Validation Error 0.916, Validation Accuracy 0.000
Epoch 51, Training Error 0.916, Validation Error 0.916, Validation Accuracy 0.000
Epoch 51, Training Error 0.915, Validation Error 0.915, Validation Accuracy 0.000
Epoch 52, Training Error 0.915, Validation Error 0.915, Validation Accuracy 0.000
Epoch 52, Training Error 0.915, Validation Error 0.915, Validation Accuracy 0.000
Epoch 52, Training Error 0.915, Validation Error 0.915, Validation Accuracy 0.000
Epoch 52, Traini

Epoch 74, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 74, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 75, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 75, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 75, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 75, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 76, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 76, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 76, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 76, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 77, Training Error 0.905, Validation Error 0.905, Validation Accuracy 0.000
Epoch 77, Training Error 0.904, Validation Error 0.904, Validation Accuracy 0.000
Epoch 77, Traini

Epoch 99, Training Error 0.902, Validation Error 0.902, Validation Accuracy 0.000
Epoch 99, Training Error 0.902, Validation Error 0.902, Validation Accuracy 0.000
Epoch 99, Training Error 0.902, Validation Error 0.902, Validation Accuracy 0.000
Time for training Neural Network using mini batches: 1.823361344577279e-06


In [19]:
pred_outputs = network.test(test_images)
error = network.calculate_error(pred_outputs, test_labels)
print(error)

0.903153644269
